# Genres Extracion Code Code

<br>
This code extracts the genres from the acquired metadata and construct an encoded array for the applicable genres for each movi. No furthr hot key encoding is needed for further work




<br>

### Importing Modules


<br>

In [1]:
import requests
import json
import time
import itertools
import wget
import os
import png
import pickle
import numpy as np
import tmdbsimple as tmdb
import imdb
import time
import sys
import urllib
from PIL import Image

import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import precision_recall_curve
import scipy

sns.set_style('white')
import tensorflow as tf
import pandas as pd
from imdb import IMDb
import ast

### Importing Files and Genres List

<br>

In [2]:
# Importing Data file
genres_raw = pd.read_csv('Data.csv').genres.values

# Importing Genres List
gen_list = pd.read_csv('genres_TMDB.csv')

# Printing a sample data
genres_raw[:5]

array(["[{u'id': 80, u'name': u'Crime'}]",
       "[{u'id': 18, u'name': u'Drama'}, {u'id': 27, u'name': u'Horror'}, {u'id': 53, u'name': u'Thriller'}]",
       "[{u'id': 35, u'name': u'Comedy'}, {u'id': 10749, u'name': u'Romance'}]",
       "[{u'id': 27, u'name': u'Horror'}]",
       "[{u'id': 35, u'name': u'Comedy'}]"], dtype=object)

In [3]:
gen_list

,Unnamed: 0,0
0,0,Action
1,1,Adventure
2,2,Animation
3,3,Comedy
4,4,Crime
5,5,Documentary
6,6,Drama
7,7,Family
8,8,Fantasy
9,9,History



<br>
### Constructing the Genres List


<br>

In [4]:
# Specify the length of the data
Len = len(genres_raw)

# Initiate the "genreslist"
genreslist = []

# Iterrate over the whole data length
for i in range (Len):
    
    # Extract the genres of the movie
    mvgenre = ast.literal_eval(genres_raw[i])
    
    

    # intitate a movie genres list
    moviegenres = []
    
    # itterrate over the length of the genres
    for j in range(len(mvgenre)):
        
        # Get the genre name    
        g = mvgenre[j]['name'].encode('utf' , 'strict')
            
        # Append
        moviegenres.append(g)
    
    # Append to the genres list
    genreslist.append(moviegenres)


In [5]:
genreslist[:10]

[['Crime'],
 ['Drama', 'Horror', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Horror'],
 ['Comedy'],
 ['Animation', 'Comedy', 'Thriller', 'Family'],
 ['Comedy', 'Family', 'Animation', 'Fantasy', 'Music'],
 ['Documentary', 'History'],
 ['Action', 'Western'],
 ['Drama']]



<br>
### Constructing the Labels Array


<br>
An array that includes a binary flag for each of the genres 

<br>

In [7]:
# Construct the genres Matrix:
# -----------------------------

# Acquire the shape of the Genres Array
shape_genre = gen_list.shape

# Intitate a multilabel array
Genres = np.zeros((Len , shape_genre[0]))

# Iterating for each movie
for i in range(Len):
    
    # Extract the list of genres dictionaries
    genr = genreslist[i]
    
    # Define the length of the labels
    Leng = len(genr)
    
    # Considering the top 3 genres
    if Leng >= 3:
        
        # Stop at the third
        Stop = 3
    
    # If shorter - take all
    else:
        
        Stop = Leng
    
    # iterrate over each dictionary of the dictionaries list
    for j in range(Stop):
        
        # Extract the movie genre name
        gename = genr[j]
        
        # Compare it to the list
        for k in range(shape_genre[0]):
            
            # Change labe if there is a match
            if gename == gen_list.iloc[k,1]:
                
                Genres[i,k] = 1

    sys.stdout.flush()
    sys.stdout.write("\r Step: {} ".format(i))

 Step: 9987 

In [8]:
# Construct the data frame:
# ------------------------

# Genres Data Frame
Genresdf = pd.DataFrame(Genres , columns = gen_list.iloc[:,1].values)

# Ids Array
Idarray = np.array(pd.read_csv('Data.csv').id.values)

# Ids Data Frame
Idframe = pd.DataFrame(Idarray , columns = ['ID'])

# Concatenate the arrays
GenresAll = pd.concat([Genresdf , Idframe] , 1)

GenresAll.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,ID
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22924
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14223
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,18307
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27420
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19157




<br>

### Exporting the Data to csv Files


<br>


In [9]:
GenresAll.to_csv('Genres_labels_All_Top3.csv')